# Track Cluster

In [226]:
import pandas as pd
import numpy as np
from utils import utils
from utils import myplot
df = pd.read_csv(r'wifi_track_data\dacang\track_data\dacang_track_data_3_final_1218.csv')
df.t = pd.to_datetime(df.t)
df_wifipos = pd.read_csv(r'wifi_track_data\dacang\pos_data\wifi_pos_new_3_1218.csv')
from tqdm import tqdm
from importlib import reload

import warnings
warnings.filterwarnings('ignore')

## 初步分析
每个date_mac的:
- 数据量
- 探针数量
- 一天内持续时间
- 一天内发生活动的平均时刻
- 一天内的总移动距离

In [227]:
mac_list = df['m'].unique()
ori_mac_list = df.oriMac.unique()
ori_Mac = []
data_count = []
Dur = []
wifi_count = []
active_moment = []
middle_moment = []
moving_dis = []
for m in mac_list:
    ori_Mac.append(m.split('-')[1])
    df_now = utils.GetDfNow(df, m)
    data_count.append(len(df_now))
    t1 = df_now.iloc[0].t
    t2 = df_now.iloc[len(df_now)-1].t
    t1 = t1.hour + t1.minute/60 + t1.second/3600
    t2 = t2.hour + t2.minute/60 + t2.second/3600
    Dur.append(t2-t1)
    middle_moment.append((t2+t1)/2)
    wifi_count.append(len(df_now['a'].unique()))
    h = 0
    row_last = df_now.iloc[0]
    dis = 0
    for i,row in df_now.iterrows():
        h += row.t.hour + row.t.minute/60 + row.t.second/3600
        if row.a != row_last.a:
            dis += utils.GetWifiTrackDistance(row.a,row_last.a,df_wifipos)
        row_last = row
    active_moment.append(h/len(df_now))
    moving_dis.append(dis)

df_sta = pd.DataFrame({'mac':mac_list,
                       'ori_mac':ori_Mac,
                       'data_count':data_count,
                       'Dur':Dur,
                       'wifi_count':wifi_count,
                       'active_moment':active_moment,
                       'moving_dis':moving_dis,
                       'middle_moment':middle_moment})
df_sta

,mac,ori_mac,data_count,Dur,wifi_count,active_moment,moving_dis,middle_moment
0,"7.5-48,74,38,133,29,213","48,74,38,133,29,213",5,7.137500,3,21.925500,255.84,20.428194
1,"7.5-48,74,38,155,214,98","48,74,38,155,214,98",9,7.140278,5,22.534414,437.00,20.429583
2,"7.5-20,178,229,137,47,142","20,178,229,137,47,142",11,0.057778,2,23.971010,697.56,23.966389
3,"7.6-48,74,38,155,214,98","48,74,38,155,214,98",7,23.999167,4,15.737460,88.12,12.000139
4,"7.6-32,50,51,210,96,61","32,50,51,210,96,61",7,23.996389,3,16.354048,84.93,11.998750
...,...,...,...,...,...,...,...,...
1270,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",14,16.207500,4,15.388036,2014.83,15.774306
1271,"8.7-124,161,119,201,50,40","124,161,119,201,50,40",32,13.351667,6,16.939488,2149.56,17.204444
1272,"8.7-56,251,20,50,26,70","56,251,20,50,26,70",5,1.326944,2,19.923944,302.32,19.667361
1273,"8.7-144,173,247,200,14,93","144,173,247,200,14,93",10,8.199167,4,18.670944,559.52,15.882361


In [228]:
myplot.Boxes([(df_sta.data_count,"数据量"),
              (df_sta.Dur,"持续时间"),
              (df_sta.wifi_count,"wifi数量"),
              (df_sta.active_moment,"活跃时刻"),
              (df_sta.moving_dis,"移动距离"),
              (df_sta.middle_moment,"中间时刻")])

## 清除离群值
随机孤立森林

In [229]:
#归一化
df_outlier = df_sta[['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']]
df_outlier = utils.Normalize_df(df_outlier,['data_count','Dur','active_moment','moving_dis','wifi_count'],mul_index=10)
df_outlier

,data_count,Dur,active_moment,moving_dis,wifi_count,middle_moment
0,0.16,2.971261,9.110291,0.365930,1.052632,20.428194
1,0.48,2.972419,9.375142,0.625044,2.105263,20.429583
2,0.64,0.020147,10.000000,0.997724,0.526316,23.966389
3,0.32,9.999884,6.418760,0.126039,1.578947,12.000139
4,0.32,9.998726,6.686949,0.121476,1.052632,11.998750
...,...,...,...,...,...,...
1270,0.88,6.752003,6.266775,2.881824,1.578947,15.774306
1271,2.32,5.561577,6.941590,3.074529,2.631579,17.204444
1272,0.16,0.549187,8.239700,0.432410,0.526316,19.667361
1273,0.56,3.413807,7.694699,0.800285,1.578947,15.882361


In [230]:
from sklearn.ensemble import IsolationForest
rng = np.random.RandomState(42)
model_IF = IsolationForest(max_samples=500,random_state=rng,contamination='auto')
model_IF.fit(df_outlier[['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']])
df_outlier['label'] = model_IF.predict(df_outlier[['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']])
print(df_outlier.label.value_counts())

myplot.Scatter_Matrix(df_outlier,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'label')

label
 1    1098
-1     177
Name: count, dtype: int64


## DBSCAN聚类

In [232]:
df_cluster = df_sta[['mac','ori_mac','data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']][df_outlier.label == 1].reset_index(drop=True)
df_cluster = utils.Normalize_df(df_cluster,['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment'],mul_index=10)

df_cluster

,mac,ori_mac,data_count,Dur,active_moment,moving_dis,wifi_count,middle_moment
0,"7.6-48,74,38,155,214,98","48,74,38,155,214,98",0.740741,9.999884,6.265394,0.258662,1.818182,4.452300
1,"7.6-32,50,51,210,96,61","32,50,51,210,96,61",0.740741,9.998726,6.580154,0.247813,0.909091,4.451525
2,"7.6-48,74,38,133,29,213","48,74,38,133,29,213",1.851852,9.998726,3.590499,1.241991,3.636364,4.451525
3,"7.6-4,207,140,11,128,186","4,207,140,11,128,186",1.296296,9.994905,2.747569,0.708242,0.909091,4.452533
4,"7.6-88,198,240,41,58,135","88,198,240,41,58,135",1.296296,9.988653,5.800856,1.285691,1.818182,4.448967
...,...,...,...,...,...,...,...,...
1093,"8.7-8,250,121,45,167,67","8,250,121,45,167,67",3.703704,6.357047,4.004173,2.906629,2.727273,6.766749
1094,"8.7-76,2,32,111,66,59","76,2,32,111,66,59",0.555556,4.143972,5.201850,1.271782,1.818182,4.788905
1095,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",2.037037,6.752003,6.087017,6.810938,1.818182,6.558910
1096,"8.7-124,161,119,201,50,40","124,161,119,201,50,40",5.370370,5.561577,6.879015,7.269122,3.636364,7.357164


In [233]:
myplot.Scatter_Matrix(df_cluster,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    'wifi_count'])

## 简单的居民、游客、路过人群分类
- 当一天内待在区域超过10小时的天数超过10天时，被视为居民
- 当一天内待在区域小于0.5小时的天数超过5天时，被视为路过
- 其余视为游客

In [234]:
def DivideId(df_now):
    days = df_now.calender.unique()
    count = 0
    isPassing = 0
    for i in range(len(days)):
        df_one = df_now[df_now.calender == days[i]]
        time = df_one.iloc[len(df_one)-1].t - df_one.iloc[0].t
        h = time.components.hours
        if h>5:
            count += 1
        elif h<0.5:
            isPassing += 1
    if isPassing > 5:
        return 'pass'
    elif count < 4:
        return 'tourist'
    else:
        return 'resident'
    

tourist_list = []
resident_list = []
pass_list = []

for mac in tqdm(ori_mac_list,desc="Dividing:"):
    df_now = df[df.oriMac == mac]
    df_now.sort_values(by='t')
    result = DivideId(df_now)
    if result == 'pass':
        pass_list.append(mac)
    elif result == 'tourist':
        tourist_list.append(mac)
    else:
        resident_list.append(mac)

# add id to df_sta
def GetId(m):
    if m in tourist_list:
        return 1
    elif m in resident_list:
        return 0
    else:
        return -1
df_sta['id'] = df_sta.ori_mac.apply(GetId)
df_cluster['id'] = df_sta.ori_mac.apply(GetId)

print('pass count:',len(pass_list))
print('tourists count:',len(tourist_list))
print('residents count:',len(resident_list))


Dividing:: 100%|██████████| 313/313 [00:01<00:00, 263.67it/s]

pass count: 0
tourists count: 237
residents count: 76


In [235]:
myplot.Scatter_Matrix(df_cluster,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'id')

## K-means++聚类

In [236]:
#获得居民最佳聚类数
df_resident = df_cluster[df_cluster.id == 0]
df_score_resi = utils.GetKmeansClusterNumDf(df_resident,['Dur','data_count','active_moment','moving_dis','middle_moment'])
myplot.Double_Axes_Line(df_score_resi,"n_clusters","calinski","silhouette","聚类数","卡林斯基-哈拉巴斯指数","轮廓系数")

In [237]:
#获得游客最佳聚类数
df_tourist = df_cluster[df_cluster.id == 1]
df_score_tour = utils.GetKmeansClusterNumDf(df_tourist,['Dur','data_count','active_moment','moving_dis','middle_moment'])
myplot.Double_Axes_Line(df_score_tour,"n_clusters","calinski","silhouette","聚类数","卡林斯基-哈拉巴斯指数","轮廓系数")

In [238]:
df_resident['cluster'] = utils.Kmeans(df_resident,['Dur','data_count','active_moment','moving_dis','middle_moment'],4)
myplot.Scatter_Matrix(df_resident,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'cluster',
                        color_scale='Jet')

In [239]:
df_tourist['cluster'] = utils.Kmeans(df_tourist,['Dur','data_count','active_moment','moving_dis','middle_moment'],3)
myplot.Scatter_Matrix(df_tourist,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'cluster',
                        color_scale='Jet')

In [240]:
df_resident.to_csv(r"wifi_track_data\dacang\cluster_data\dacang_track_data_resident.csv",index=False)
df_tourist.to_csv(r"wifi_track_data\dacang\cluster_data\dacang_track_data_tourist.csv",index=False)